In [1]:
# ニューラルネットワークを使って予測してみる

import pandas as pd
import numpy as np
import tensorflow as tf
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# データの読み込み
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

# データの確認
display(train.dtypes)
display(test.dtypes)

# データ全体の確認
display(train.describe())
display(test.describe())

# 欠損データの確認
kesson_train = train.isnull().sum()
kesson_test = test.isnull().sum()
display(kesson_train)
display(kesson_test)


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [2]:
# 名前から敬称を抽出、家族サイズの作成、AgeとFareをカテゴリ化する関数
def feature_engineering(df):
    # 敬称の抽出
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

    # 家族サイズの作成、カテゴリ化
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

    df["FamilySizeGroup"] = "Small"
    df.loc[df["FamilySizeGroup"] == 1, "FamilySizeGroup"] = "Solo"
    df.loc[(df["FamilySize"] >= 5), "FamilySizeGroup"] = "Large"

    # Age、Fareのカテゴリ化
    age_fill = df['Age'].fillna(df["Age"].median())
    df["AgeGroup"] = pd.cut(age_fill.astype(int), 5, labels=['Child', 'Young', 'Adult', 'Mid-age', 'Senior'])

    df_fill = df['Fare'].fillna(df["Fare"].mean())
    df["FareGroup"] = pd.cut(df_fill, 4, labels=['Low', 'Mid', 'High', 'Very high'])

    return df

# 特徴量エンジニアリングの実行
train = feature_engineering(train)
test = feature_engineering(test)

# パイプラインの作成
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# ColumnTransformerの作成
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, [
            "Title",
            "FamilySizeGroup",
            "AgeGroup",
            "FareGroup",
            "Sex",
            "Embarked",
            "Pclass"
        ]),
    ]
)

# 前処理実行
y_train = train["Survived"]

X_train_preprocessed = preprocessor.fit_transform(train)
X_test_preprocessed = preprocessor.transform(test)

display(X_train_preprocessed.shape)
display(X_train_preprocessed[:5])

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
C:\Users\kuon9\AppData\Local\Temp\ipykernel_43796\4256088367.py:4: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


(891, 24)

array([[0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        1., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        1., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 0., 0., 1.]])

In [3]:
# ニューラルネットワークのモデル作成関数
def create_model(units=32, dropout_rate=0.3, learning_rate=0.001):
    model = Sequential([
        Dense(units, activation="relu", input_shape=(X_train_preprocessed.shape[1],)),
        Dropout(dropout_rate),
        Dense(units // 2, activation="relu"),
        Dropout(dropout_rate),
        Dense(1, activation="sigmoid")
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

# early stoppingの設定
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    mode='max',
    restore_best_weights=True
)

# KerasClassifierのラッパーの作成
model = KerasClassifier(
    model=create_model,
    validation_split=0.2,
    units=32,
    dropout_rate=0.3,
    learning_rate=0.001,
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping]
)

# 探索するハイパーパラメータの範囲を定義
param = {
    "model__units": [16, 32, 64, 128],
    "model__dropout_rate": [0.2, 0.3, 0.4, 0.5],
    "model__learning_rate": [0.001, 0.0005],
    "batch_size": [16, 32, 64],
    "epochs": [100]
}

# RandomizedSearchCVの設定
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    verbose=1,
)

# ハイパーパラメータ探索の実行
search_result = random_search.fit(
    X_train_preprocessed,
    y_train,
    callbacks=[early_stopping]
)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Epoch 1/100


c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7389 - loss: 0.5804 - val_accuracy: 0.8067 - val_loss: 0.4765
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8126 - loss: 0.4661 - val_accuracy: 0.8403 - val_loss: 0.4039
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8253 - loss: 0.4284 - val_accuracy: 0.8487 - val_loss: 0.3915
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8189 - loss: 0.4046 - val_accuracy: 0.8403 - val_loss: 0.3754
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8063 - loss: 0.3941 - val_accuracy: 0.8824 - val_loss: 0.3579
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8316 - loss: 0.3839 - val_accuracy: 0.8655 - val_loss: 0.3523
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8274 - loss: 0.3830 - val_accuracy: 0.8739 - val_loss: 0.3623
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8358 - loss: 0.3779 - val_accuracy: 0.8908 - val_loss: 0.3

c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6358 - loss: 0.6153 - val_accuracy: 0.7983 - val_loss: 0.5214
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7705 - loss: 0.5049 - val_accuracy: 0.8151 - val_loss: 0.4344
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7937 - loss: 0.4645 - val_accuracy: 0.8487 - val_loss: 0.4021
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8000 - loss: 0.4519 - val_accuracy: 0.8571 - val_loss: 0.3986
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8105 - loss: 0.4316 - val_accuracy: 0.8487 - val_loss: 0.3884
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8147 - loss: 0.4382 - val_accuracy: 0.8487 - val_loss: 0.3723
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8189 - loss: 0.4120 - val_accuracy: 0.8487 - val_loss: 0.3637
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8295 - loss: 0.4079 - val_accuracy: 0.8571 - val_loss: 0.3

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6147 - loss: 0.6455 - val_accuracy: 0.6471 - val_loss: 0.5788
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7011 - loss: 0.5872 - val_accuracy: 0.7815 - val_loss: 0.5164
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7705 - loss: 0.5123 - val_accuracy: 0.7983 - val_loss: 0.4591
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8105 - loss: 0.5001 - val_accuracy: 0.8487 - val_loss: 0.4266
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8021 - loss: 0.4838 - val_accuracy: 0.8487 - val_loss: 0.4061
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7979 - loss: 0.4567 - val_accuracy: 0.8319 - val_loss: 0.3944
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8084 - loss: 0.4485 - val_accuracy: 0.8487 - val_loss: 0.3868
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8316 - loss: 0.4438 - val_accuracy: 0.8487 - val_loss: 0.3

c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6232 - loss: 0.6548 - val_accuracy: 0.6975 - val_loss: 0.5900
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7453 - loss: 0.5701 - val_accuracy: 0.7983 - val_loss: 0.5279
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8105 - loss: 0.5087 - val_accuracy: 0.8067 - val_loss: 0.4895
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8063 - loss: 0.4773 - val_accuracy: 0.7983 - val_loss: 0.4782
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8126 - loss: 0.4678 - val_accuracy: 0.8067 - val_loss: 0.4706
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8211 - loss: 0.4654 - val_accuracy: 0.8067 - val_loss: 0.4629
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8189 - loss: 0.4544 - val_accuracy: 0.8151 - val_loss: 0.4559
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8189 - loss: 0.4388 - val_accuracy: 0.8151 - val_loss: 0.4

c:\Users\kuon9\Documents\Kaggle\titanic\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7289 - loss: 0.5476 - val_accuracy: 0.8212 - val_loss: 0.4243
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8090 - loss: 0.4549 - val_accuracy: 0.8436 - val_loss: 0.3840
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8188 - loss: 0.4353 - val_accuracy: 0.8771 - val_loss: 0.3521
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8244 - loss: 0.4287 - val_accuracy: 0.8771 - val_loss: 0.3415
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8188 - loss: 0.4209 - val_accuracy: 0.8939 - val_loss: 0.3339
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8230 - loss: 0.4189 - val_accuracy: 0.8827 - val_loss: 0.3448
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8287 - loss: 0.4221 - val_accuracy: 0.8659 - val_loss: 0.3379
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8315 - loss: 0.4112 - val_accuracy: 0.8939 - val_loss: 0.3

In [4]:
# 最良モデルの情報を表示
print(f"Best Score: {search_result.best_score_:.4f}")
print(f"Best Params: {search_result.best_params_}")

best_model = search_result.best_estimator_

Best Score: 0.8272
Best Params: {'model__units': 128, 'model__learning_rate': 0.0005, 'model__dropout_rate': 0.2, 'epochs': 100, 'batch_size': 16}


In [5]:
# テストデータで予測
probabilities = best_model.predict_proba(X_test_preprocessed)[:, 1]
predictions = np.round(probabilities).astype(int).flatten()

# 提出用ファイル作成
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions
})

submission.to_csv("titanic_NN_solution.csv", index=False)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
